# **Instructions** 
---
### Execution Environment 

This notebook is designed to be executed in Google Colaboratory (Colab). Running it in other environments might require modifications to the code.


### Google Earth Engine Account

This notebook utilizes Google Earth Engine. Make sure you have an active Google Earth Engine Account and access to the Google Earth Engine Console.

### Google Drive Storage
The output files will be saved to your linked Google Drive account. Please verify that you have sufficient storage space available. Remember, large datasets may consume significant storage.

---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# install dependencies
!pip install rasterio >> /dev/null
!pip install geopandas >> /dev/null

In [44]:
from pydantic import BaseModel
import pandas as pd 
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Polygon, Point
from math import cos, radians
import os
import rasterio
import numpy as np
import glob
import ee
from datetime import date
from datetime import datetime
import math
import logging
import time

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
%matplotlib inline 

In [45]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=qMFjLo8IrGoWqFpAeYmMFv6awIcdThQqzCQNE5hOIyk&tc=MeeM6RYxBJpKQOxJ0t600frUpyNSzi006XmdlWEaonE&cc=gbDT3S2n6-D7ge1bu33Uji7QdBBMH3YSmfklwJx8PfA

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VNcUMAWwlyLygkLDKgogD4WBf9b_acn6DxArWPaKRCS6BFTXS_n9OU

Successfully saved authorization token.


# 01 - Pre-Process Data

In [46]:
# Define data
data = {
    'LOCALITY ID': ['L2522641', 'L915651', 'L915651'],
    'LATITUDE': [-1.276791, -1.368354, -1.368354],
    'LONGITUDE': [36.810722, 36.850891, 36.850891]
}

# Create DataFrame
data = pd.DataFrame(data)


In [47]:
data.head()

,LOCALITY ID,LATITUDE,LONGITUDE
0,L2522641,-1.276791,36.810722
1,L915651,-1.368354,36.850891
2,L915651,-1.368354,36.850891


In [49]:
# Extract the geometry from the lat/lon 
geometry = [Point(xy) for xy in zip(data['LONGITUDE'], data['LATITUDE'])]

In [50]:
# Convert the data into a geo_dataframae
crs = {'init':'epsg:4326'}
geo_df = gpd.GeoDataFrame(data,
                          crs=crs,
                          geometry=geometry)

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [51]:
geo_df

,LOCALITY ID,LATITUDE,LONGITUDE,geometry
0,L2522641,-1.276791,36.810722,POINT (36.81072 -1.27679)
1,L915651,-1.368354,36.850891,POINT (36.85089 -1.36835)
2,L915651,-1.368354,36.850891,POINT (36.85089 -1.36835)


In [52]:
def generate_buffer_meter(data, radiu, geometry='geometry', crs='epsg:4326'):
    """
    Generates a buffer around the geometries in a geopandas DataFrame.

    Parameters:
    data (GeoDataFrame or DataFrame): The geopandas dataframe or a pandas dataframe that contains geometry data.
    radiu (float): The radius of the buffer in meters.
    geometry (str, optional): The column in the dataframe that contains the geometry information. 
                              Defaults to 'geometry'.
    crs (str, optional): The Coordinate Reference System to be used. Defaults to 'epsg:4326'.

    Returns:
    GeoDataFrame: A new geopandas dataframe with the buffer applied to the geometry.
    """

    data = gpd.GeoDataFrame(data)
    data = data.to_crs(crs)
    data = data.to_crs('+proj=aeqd +units=m  +x_0=0 +y_0=0')
    data[geometry] = data[geometry].buffer(radiu, cap_style=3)
    data = data.to_crs(crs)

    return data


In [53]:
# Generate a buffer of 3km around the geometries in a geopandas DataFrame
data_df = generate_buffer_meter(geo_df, 3000)

In [54]:
data_df.head()

,LOCALITY ID,LATITUDE,LONGITUDE,geometry
0,L2522641,-1.276791,36.810722,"POLYGON ((36.83742 -1.25137, 36.83793 -1.30195..."
1,L915651,-1.368354,36.850891,"POLYGON ((36.87757 -1.34293, 36.87812 -1.39349..."
2,L915651,-1.368354,36.850891,"POLYGON ((36.87757 -1.34293, 36.87812 -1.39349..."


#02 -  Loading Data from Earth Engine

In [55]:
def get_single_image(region: ee.Geometry, start_date: date, end_date: date) -> ee.Image:

    dates = ee.DateRange(date_to_string(start_date), date_to_string(end_date))

    startDate = ee.DateRange(dates).start()
    endDate = ee.DateRange(dates).end()
    imgC = ee.ImageCollection(image_collection).filterDate(startDate, endDate).filterBounds(region)

    imgC = (
        imgC.map(lambda x: x.clip(region))
        .map(lambda x: x.set("ROI", region))
        .map(computeS2CloudScore)
        .map(projectShadows)
        .map(computeQualityScore)
        .sort("CLOUDY_PIXEL_PERCENTAGE")
    )

    # has to be double to be compatible with the sentinel 1 imagery, which is in
    # float64
    cloudFree = mergeCollection(imgC).toDouble()

    return cloudFree

def get_single_image_landuse(region: ee.Geometry, start_date: date, end_date: date) -> ee.Image:

    dates = ee.DateRange(date_to_string(start_date), date_to_string(end_date))

    startDate = ee.DateRange(dates).start()
    endDate = ee.DateRange(dates).end()
    imgC = ee.ImageCollection(glcd).filterDate(startDate, endDate).filterBounds(region)

    imgC = (
        imgC.map(lambda x: x.clip(region))
    )

    output = merge_lc_collection(imgC)

    return output


def rescale(img, exp, thresholds):
    return (
        img.expression(exp, {"img": img})
        .subtract(thresholds[0])
        .divide(thresholds[1] - thresholds[0])
    )


def computeQualityScore(img):
    score = img.select(["cloudScore"]).max(img.select(["shadowScore"]))

    score = score.reproject("EPSG:4326", None, 20).reduceNeighborhood(
        reducer=ee.Reducer.mean(), kernel=ee.Kernel.square(5), optimization="boxcar"
    )

    score = score.multiply(-1)

    return img.addBands(score.rename("cloudShadowScore"))


def computeS2CloudScore(img):
    toa = img.select(
        ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9", "B10", "B11", "B12"]
    ).divide(10000)

    toa = toa.addBands(img.select(["QA60"]))

    # ['QA60', 'B1','B2',    'B3',    'B4',   'B5','B6','B7', 'B8','  B8A',
    #  'B9',          'B10', 'B11','B12']
    # ['QA60','cb', 'blue', 'green', 'red', 're1','re2','re3','nir', 'nir2',
    #  'waterVapor', 'cirrus','swir1', 'swir2']);

    # Compute several indicators of cloudyness and take the minimum of them.
    score = ee.Image(1)

    # Clouds are reasonably bright in the blue and cirrus bands.
    score = score.min(rescale(toa, "img.B2", [0.1, 0.5]))
    score = score.min(rescale(toa, "img.B1", [0.1, 0.3]))
    score = score.min(rescale(toa, "img.B1 + img.B10", [0.15, 0.2]))

    # Clouds are reasonably bright in all visible bands.
    score = score.min(rescale(toa, "img.B4 + img.B3 + img.B2", [0.2, 0.8]))

    # Clouds are moist
    ndmi = img.normalizedDifference(["B8", "B11"])
    score = score.min(rescale(ndmi, "img", [-0.1, 0.1]))

    # However, clouds are not snow.
    ndsi = img.normalizedDifference(["B3", "B11"])
    score = score.min(rescale(ndsi, "img", [0.8, 0.6]))

    # Clip the lower end of the score
    score = score.max(ee.Image(0.001))

    # score = score.multiply(dilated)
    score = score.reduceNeighborhood(reducer=ee.Reducer.mean(), kernel=ee.Kernel.square(5))

    return img.addBands(score.rename("cloudScore"))


def projectShadows(image):
    meanAzimuth = image.get("MEAN_SOLAR_AZIMUTH_ANGLE")
    meanZenith = image.get("MEAN_SOLAR_ZENITH_ANGLE")

    cloudMask = image.select(["cloudScore"]).gt(cloudThresh)

    # Find dark pixels
    darkPixelsImg = image.select(["B8", "B11", "B12"]).divide(10000).reduce(ee.Reducer.sum())

    ndvi = image.normalizedDifference(["B8", "B4"])
    waterMask = ndvi.lt(ndviThresh)

    darkPixels = darkPixelsImg.lt(irSumThresh)

    # Get the mask of pixels which might be shadows excluding water
    darkPixelMask = darkPixels.And(waterMask.Not())
    darkPixelMask = darkPixelMask.And(cloudMask.Not())

    # Find where cloud shadows should be based on solar geometry
    # Convert to radians
    azR = ee.Number(meanAzimuth).add(180).multiply(math.pi).divide(180.0)
    zenR = ee.Number(meanZenith).multiply(math.pi).divide(180.0)

    # Find the shadows
    def getShadows(cloudHeight):
        cloudHeight = ee.Number(cloudHeight)
        
        shadowCastedDistance = zenR.tan().multiply(cloudHeight)  # Distance shadow is cast
        x = azR.sin().multiply(shadowCastedDistance).multiply(-1)  # /X distance of shadow
        y = azR.cos().multiply(shadowCastedDistance).multiply(-1)  # Y distance of shadow
        return image.select(["cloudScore"]).displace(
            ee.Image.constant(x).addBands(ee.Image.constant(y))
        )

    shadows = ee.List(cloudHeights).map(getShadows)
    shadowMasks = ee.ImageCollection.fromImages(shadows)
    shadowMask = shadowMasks.mean()

    # Create shadow mask
    shadowMask = dilatedErossion(shadowMask.multiply(darkPixelMask))

    shadowScore = shadowMask.reduceNeighborhood(
        **{"reducer": ee.Reducer.max(), "kernel": ee.Kernel.square(1)}
    )

    image = image.addBands(shadowScore.rename(["shadowScore"]))

    return image


def dilatedErossion(score):
    # Perform opening on the cloud scores

    def erode(img, distance):
        d = (
            img.Not()
            .unmask(1)
            .fastDistanceTransform(30)
            .sqrt()
            .multiply(ee.Image.pixelArea().sqrt())
        )
        return img.updateMask(d.gt(distance))

    def dilate(img, distance):
        d = img.fastDistanceTransform(30).sqrt().multiply(ee.Image.pixelArea().sqrt())
        return d.lt(distance)

    score = score.reproject("EPSG:4326", None, 20)
    score = erode(score, erodePixels)
    score = dilate(score, dilationPixels)

    return score.reproject("EPSG:4326", None, 20)


def mergeCollection(imgC):
    filtered = imgC.qualityMosaic("cloudShadowScore")
    filtered = filtered.select(BANDS)
    return filtered

def merge_lc_collection(imgC):
    filtered = imgC.qualityMosaic("cloudShadowScore")
    filtered = filtered.select(lc_bands)
    return filtered


def date_to_string(input_date):
    if isinstance(input_date, str):
        return input_date
    else:
        assert isinstance(input_date, date)
        return input_date.strftime("%Y-%m-%d")

In [56]:
def download_sentinel2_data(data_df, start_date, end_date, drive_folder, scale=10, max_pixels=1e13):
    """
    Downloads Sentinel 2 data for unique locations from the given DataFrame.

    Parameters:
    data_df (GeoDataFrame): A GeoDataFrame that contains 'geometry' and 'LOCALITY ID' columns.
    start_date (str): The start date for the image collection in the format 'YYYY-MM-DD'.
    end_date (str): The end date for the image collection in the format 'YYYY-MM-DD'.
    drive_folder (str): The Google Drive folder to export the images to.
    scale (int, optional): The scale to use when exporting the image. Default is 10.
    max_pixels (int or float, optional): The maximum allowed number of pixels in the exported image. Default is 1e13.
    
    Returns:
    None
    """
    # Error handling for inputs
    if data_df.empty:
        raise ValueError("Input dataframe is empty")
    if not {'geometry', 'LOCALITY ID'}.issubset(data_df.columns):
        raise ValueError("Input dataframe should contain 'geometry' and 'LOCALITY ID' columns")
    if not datetime.strptime(start_date, '%Y-%m-%d') < datetime.strptime(end_date, '%Y-%m-%d'):
        raise ValueError("Start date should be before end date")

    # Iterate over each row in the DataFrame
    for index, row in data_df.iterrows():
        # Log index every 20 iterations
        if index % 20 == 0:
            logging.info(f"Processing index: {index}")

        # Get coordinates and create a polygon
        coords = data_df["geometry"][index].exterior.coords
        contour = list((np.asarray(coords)).flatten())
        region = create_region(contour)

        # Get the image
        img = get_single_image(region, start_date, end_date)

        # Create export task
        task = ee.batch.Export.image(
            img.clip(region),
            data_df["LOCALITY ID"][index],
            {"scale": scale, "region": region, "maxPixels": max_pixels, "driveFolder": drive_folder},
        )
        task.start()


def create_region(contour):
    """
    Creates a polygon from a contour.

    Parameters:
    contour (list): A list of coordinates.

    Returns:
    region (ee.Geometry.Polygon): A polygon that represents the region.
    """
    return ee.Geometry.Polygon(contour)

def list_ee_tasks():
    print("Running tasks:")
    running_tasks = [task for task in ee.batch.Task.list() if task.state == ee.batch.Task.State.RUNNING]
    for task in running_tasks:
        print(task.id)

    print("Completed tasks:")
    completed_tasks = [task for task in ee.batch.Task.list() if task.state == ee.batch.Task.State.COMPLETED]
    for task in completed_tasks:
        print(task.id)

def check_ee_tasks():
    # Get a list of all tasks
    tasks = ee.batch.Task.list()

    # Filter tasks that are running
    running_tasks = [task for task in tasks if task.state == ee.batch.Task.State.RUNNING]
    running_count = len(running_tasks)

    # Filter tasks that are completed
    completed_tasks = [task for task in tasks if task.state == ee.batch.Task.State.COMPLETED]
    completed_count = len(completed_tasks)

    # Filter tasks that have failed
    failed_tasks = [task for task in tasks if task.state == ee.batch.Task.State.FAILED]
    failed_count = len(failed_tasks)

    # Log the status
    logger.info(f"Running: {running_count}, Completed: {completed_count}, Failed: {failed_count}")

# 03 - Finally Download the Data

In [58]:
#  These are algorithm settings for the cloud filtering algorithm
image_collection = "COPERNICUS/S2"

# Ranges from 0-1.Lower value will mask more pixels out.
# Generally 0.1-0.3 works well with 0.2 being used most commonly
cloudThresh = 0.2

# Height of clouds to use to project cloud shadows
cloudHeights = [200, 10000, 250]

# Sum of IR bands to include as shadows within TDOM and the
# shadow shift method (lower number masks out less)
irSumThresh = 0.3
ndviThresh = -0.1

# Pixels to reduce cloud mask and dark shadows by to reduce inclusion
# of single-pixel comission errors
erodePixels = 1.5
dilationPixels = 3

# images with less than this many cloud pixels will be used with normal
# mosaicing (most recent on top)
cloudFreeKeepThresh = 3

# Bands to Download
BANDS = ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9", "B10", "B11", "B12"]
lc_bands = ["water", "trees", "grass"]

# Define the start date, end date and the Google Drive folder name
start_date = "2015-06-01"
end_date = "2020-06-30"
drive_folder = "test_dir_2"

In [59]:

# Call the function
download_sentinel2_data(data_df, start_date, end_date, drive_folder)

INFO:root:Processing index: 0
2023-05-18 20:52:15,237 - INFO - Processing index: 0


In [ ]:
# super optional: indefinitely log progress for sanity purposes :)
# keep in mind the function will run indefinitely,
#  so you need to manually stop it.
while True:
    check_ee_tasks()
    time.sleep(60)  # Wait for 60 seconds

INFO:__main__:Running: 2, Completed: 4, Failed: 0
2023-05-18 20:52:24,720 - INFO - Running: 2, Completed: 4, Failed: 0
INFO:__main__:Running: 4, Completed: 5, Failed: 0
2023-05-18 20:53:25,884 - INFO - Running: 4, Completed: 5, Failed: 0
INFO:__main__:Running: 4, Completed: 5, Failed: 0
2023-05-18 20:54:26,985 - INFO - Running: 4, Completed: 5, Failed: 0
INFO:__main__:Running: 4, Completed: 5, Failed: 0
2023-05-18 20:55:28,093 - INFO - Running: 4, Completed: 5, Failed: 0
INFO:__main__:Running: 3, Completed: 6, Failed: 0
2023-05-18 20:56:29,342 - INFO - Running: 3, Completed: 6, Failed: 0
INFO:__main__:Running: 3, Completed: 6, Failed: 0
2023-05-18 20:57:30,534 - INFO - Running: 3, Completed: 6, Failed: 0
INFO:__main__:Running: 3, Completed: 6, Failed: 0
2023-05-18 20:58:31,710 - INFO - Running: 3, Completed: 6, Failed: 0
INFO:__main__:Running: 3, Completed: 6, Failed: 0
2023-05-18 20:59:32,939 - INFO - Running: 3, Completed: 6, Failed: 0
INFO:__main__:Running: 3, Completed: 6, Failed: 